In [1]:
from utils import calculate_right_top_coordinates, calculate_one_pixels_diffs
from google_api_load_data import load_prepare_and_cut_image
from openstreetmap_load_data import load_data_from_openstreetmap, find_objects_bounds

In [2]:
import cv2
import numpy as np

In [3]:
left_bottom_lat = 54.4081
left_bottom_lon = 48.980
right_top_lat, right_top_lon = calculate_right_top_coordinates(left_bottom_lat, left_bottom_lon)
print(right_top_lat, right_top_lon)

54.647034467866604 49.529291338582674


In [192]:
# img_cutted_satellite = load_prepare_and_cut_image(left_bottom_lat=left_bottom_lat, left_bottom_lon=left_bottom_lon,
#                                                   right_top_lat=right_top_lat, right_top_lon=right_top_lon,
#                                                   is_debug=True)

In [5]:
img_cutted_satellite.shape

(300, 400, 3)

In [6]:
# Как обычно проблемы с соединением...
load_data_from_openstreetmap(left_bottom_lat, left_bottom_lon, right_top_lat, right_top_lon, "tmp.json")

In [7]:

datafile_name = "tmp.json"

bounds, tags = find_objects_bounds(datafile_name, right_top_lat, right_top_lon, left_bottom_lat, left_bottom_lon)

In [8]:
bounds[0]

{'maxlat': 54.647034467866604,
 'minlat': 54.6400339,
 'maxlon': 48.985209,
 'minlon': 48.98}

In [9]:
tags[0]

'water'

In [15]:
class One_OSM_object(object):
    """
    Представляет границы одного объекта на изображении в пикселях
    """
    def __init__(self, maxlat, maxlon, minlat, minlon, tag):
        self.maxlat = maxlat
        self.maxlon = maxlon
        self.minlat = minlat
        self.minlon = minlon
        self.tag = tag
    
    def print_osm_object(self):
        print("maxlat: %d, minlat: %d, maxlon: %d, minlon: %d" % (self.maxlat, self.minlat, self.maxlon, self.minlon))

class Image_OSM_object(object):
    """
    Класс, умеющий получать из данных osm и хранить границы объектов на изображении в пикселях
    """
    
    def __init__(self, img_cutted_shape, right_top_lat, right_top_lon, left_bottom_lat, left_bottom_lon, bounds, tags):
        self.img_cutted_shape = img_cutted_shape
        self.right_top_lat = right_top_lat
        self.right_top_lon = right_top_lon
        self.left_bottom_lat = left_bottom_lat
        self.left_bottom_lon = left_bottom_lon
        self.one_pixel_lat_diff, self.one_pixel_lon_diff = calculate_one_pixels_diffs(right_top_lat, right_top_lon,
                                                                                      left_bottom_lat, left_bottom_lon,
                                                                                      img_cutted_shape)
        self.osm_objects = []
        for i in range(len(bounds)):
            self.calculate_pixel_bounds(bounds[i], tags[i])
        
    def calculate_pixel_bounds(self, grads_bounds, tag):        
        maxlat_grads = grads_bounds['maxlat']
        maxlon_grads = grads_bounds['maxlon']
        minlat_grads = grads_bounds['minlat']
        minlon_grads = grads_bounds['minlon']
        
        minlat_pixel = self.img_cutted_shape[0] - int(round((minlat_grads - self.left_bottom_lat) / self.one_pixel_lat_diff, 0))
        maxlat_pixel = int(round((self.right_top_lat - maxlat_grads) / self.one_pixel_lat_diff, 0))
        minlon_pixel = int(round((minlon_grads - self.left_bottom_lon) / self.one_pixel_lon_diff, 0))
        maxlon_pixel = self.img_cutted_shape[1] - int(round((self.right_top_lon - maxlon_grads) / self.one_pixel_lon_diff, 0))
        
        maxlat_pixel = np.clip(maxlat_pixel, 0, self.img_cutted_shape[0] - 1)
        minlat_pixel = np.clip(minlat_pixel, 0, self.img_cutted_shape[0] - 1)
        maxlon_pixel = np.clip(maxlon_pixel, 0, self.img_cutted_shape[1] - 1)
        minlon_pixel = np.clip(minlon_pixel, 0, self.img_cutted_shape[1] - 1)
        
        if minlat_pixel != maxlat_pixel and minlon_pixel != maxlon_pixel\
            and (minlat_pixel - maxlat_pixel) > self.img_cutted_shape[0] * 0.01\
            and (maxlon_pixel - minlon_pixel) > self.img_cutted_shape[1] * 0.01\
            and (minlat_pixel - maxlat_pixel) < self.img_cutted_shape[0] * 0.99\
            and (minlat_pixel - maxlat_pixel) < self.img_cutted_shape[1] * 0.99:
            self.osm_objects.append(One_OSM_object(maxlat_pixel, maxlon_pixel, minlat_pixel, minlon_pixel, tag))
            
        return

In [16]:
img_osm_1 = Image_OSM_object(img_cutted_satellite.shape, right_top_lat, right_top_lon,
                             left_bottom_lat, left_bottom_lon, bounds, tags)

In [17]:
img_osm_1.osm_objects[8].print_osm_object()

maxlat: 0, minlat: 250, maxlon: 399, minlon: 104


In [18]:
len(img_osm_1.osm_objects)

135

Теперь надо научиться строить круговую диаграмму. Начнём с одной круговой диаграммы для конкретной точки и даже (для начала) конкретного объекта.

In [183]:
def calculate_angle(vector_1, vector_2):
    scalar_product = np.dot(vector_1, vector_2)
    norm_1 = np.linalg.norm(vector_1)
    norm_2 = np.linalg.norm(vector_2) 
    return np.rad2deg(np.arccos(scalar_product / (norm_1 * norm_2)))


def calculate_directed_angle(vector_1, vector_2):
    
    angle = calculate_angle(vector_1, vector_2)
    
#     print("dir angle input:", vector_1, vector_2)
    pseudoscalar_product = vector_1[0] * vector_2[1] - vector_1[1] * vector_2[0]
    norm_1 = np.linalg.norm(vector_1)
    norm_2 = np.linalg.norm(vector_2) 
    dir_angle = np.rad2deg(np.arcsin(pseudoscalar_product / (norm_1 * norm_2)))
    
    if dir_angle < 0:
        angle = 360 - angle
#     print("dir angle result:", angle)
    return angle    


def calculate_border_angles_to_object_and_point(x, y, osm_object):
    """
    Функция, вычисляющая для одного osm-объекта два угла, между которыми можно видеть объект, находясь в точке (x, y)
    """
    minlat = osm_object.minlat
    maxlat = osm_object.maxlat
    minlon = osm_object.minlon
    maxlon = osm_object.maxlon

    # Получаем 4 вектора для каждой угловой точки границы объекта
    v1 = (maxlat - x, minlon - y)
    v2 = (maxlat - x, maxlon - y)
    v3 = (minlat - x, minlon - y)
    v4 = (minlat - x, maxlon - y)
    
    # Для каждой пары векторов посчитаем угол между ними
    vectors = [v1, v2, v3, v4]
    print(vectors)
    angle_vector_pair = {}
    for i in range(4):
        for j in range(i, 4):
            if i != j:
                vector_1 = vectors[i]
                vector_2 = vectors[j]

                angle = abs(calculate_angle(vector_1, vector_2))
                angle_vector_pair[angle] = (i, j)
#     print(angle_vector_pair)
                
    # Теперь нужно определить, какие 2 вектора дают максимальный угол
    max_angle = max(angle_vector_pair)
    vector_pair = angle_vector_pair[max_angle]
#     print(vector_pair)
    vector_1 = vectors[vector_pair[0]]
    vector_2 = vectors[vector_pair[1]]

    # Определяем угол между каждым из векторов и осью абсцисс, начинающейся в точке (x, y)
    # (x - x, right_top_lon_pix - y)
    abscissa = (0, img_cutted_satellite.shape[1] - y)
#     print("abscissa:", abscissa)
    
    angle_1 = calculate_directed_angle(abscissa, vector_1)
    angle_2 = calculate_directed_angle(abscissa, vector_2)
    
    return angle_1, angle_2 

In [184]:
class Circle_diagram(object):
    def __init__(self, sectors_count):
        self.sectors_count = sectors_count
    
        # Количество градусов, которое помещается в каждом секторе
        self.step_grad = 360 / sectors_count
        
        self.sectors = [[] for _ in range(sectors_count)]
        
    def insert(self, angle_1, angle_2, tag):
        diff_between_angles = abs(angle_2 - angle_1)
        if diff_between_angles < 180: # Перехода через 0 нет
            start_angle = min(angle_1, angle_2)
            end_angle = max(angle_1, angle_2)
            
        if diff_between_angles >= 180: # Есть переход через 0
            start_angle = max(angle_1, angle_2)
            end_angle = min(angle_1, angle_2)
            
        sector_start_idx = int(start_angle / self.step_grad)
#         print(sector_start_idx)
        sector_end_idx = int(end_angle / self.step_grad)
#         print(sector_end_idx)
        
        if diff_between_angles < 180:
            for sector_idx in range(sector_start_idx, sector_end_idx + 1):    
                self.sectors[sector_idx].append(tag)
        else:
            for sector_idx in list(range(sector_start_idx, self.sectors_count)) + list(range(0, sector_end_idx)):
                self.sectors[sector_idx].append(tag)
                

In [161]:
cd = Circle_diagram(8)
cd.step_grad

45.0

In [118]:
cd.insert(calculate_angle(abscissa, vector_1), calculate_angle(abscissa, vector_2), tag)

0
0


In [119]:
cd.sectors

[['water'], [], [], [], [], [], [], []]

In [ ]:
# Внимательно протестировать случай, где есть переход через 0

In [187]:
x = 200
y = 200

cd = Circle_diagram(8)

for i in range(len(img_osm_1.osm_objects)):

    osm_object = img_osm_1.osm_objects[i]

    angle_1, angle_2 = calculate_border_angles_to_object_and_point(x, y, osm_object)

    cd.insert(angle_1, angle_2, osm_object.tag)
    print(cd.sectors)

[(-175, -116), (-175, -84), (-160, -116), (-160, -84)]
[[], [], ['water'], [], [], [], [], []]
[(49, -27), (49, -18), (59, -27), (59, -18)]
[[], [], ['water'], [], [], ['water'], [], []]
[(86, 145), (86, 152), (97, 145), (97, 152)]
[[], [], ['water'], [], [], ['water'], [], ['water']]
[(-137, -195), (-137, -188), (-129, -195), (-129, -188)]
[[], [], ['water'], ['water'], [], ['water'], [], ['water']]
[(-145, -181), (-145, -160), (-135, -181), (-135, -160)]
[[], [], ['water'], ['water', 'water'], [], ['water'], [], ['water']]
[(-164, -96), (-164, -91), (-157, -96), (-157, -91)]
[[], [], ['water', 'water'], ['water', 'water'], [], ['water'], [], ['water']]
[(-155, -179), (-155, -148), (-144, -179), (-144, -148)]
[[], [], ['water', 'water', 'wood'], ['water', 'water', 'wood'], [], ['water'], [], ['water']]
[(-200, -200), (-200, -94), (-164, -200), (-164, -94)]
[[], [], ['water', 'water', 'wood', 'wood'], ['water', 'water', 'wood', 'wood'], [], ['water'], [], ['water']]
[(-200, -96), (-200

In [151]:
img_osm_1.osm_objects[7].print_osm_object()

maxlat: 0, minlat: 36, maxlon: 106, minlon: 0


In [180]:
200 - 40

160

In [179]:
200 - 84

116

In [190]:
cd.sectors

[['wood'],
 ['wood',
  'wood',
  'wood',
  'wood',
  'wood',
  'wood',
  'wood',
  'wood',
  'wood',
  'wood',
  'wood',
  'wood',
  'wood',
  'wood',
  'wood',
  'wood',
  'wood',
  'wood',
  'wood',
  'wood',
  'wood',
  'wood',
  'wood',
  'wood',
  'wood',
  'wood',
  'wood',
  'wood',
  'wood',
  'wood',
  'wood',
  'wood',
  'wood',
  'wood',
  'wood',
  'wood',
  'wood',
  'wood',
  'wood',
  'wood',
  'wood',
  'wood',
  'wood',
  'wood',
  'wood',
  'wood',
  'wood',
  'wood',
  'wood',
  'wood',
  'wood',
  'wood',
  'wood',
  'wood',
  'wood',
  'wood',
  'wood',
  'wood',
  'wood',
  'wood',
  'wood',
  'wood',
  'wood',
  'wood',
  'wood',
  'wood',
  'wood',
  'wood',
  'wood',
  'wood'],
 ['water',
  'water',
  'wood',
  'wood',
  'wood',
  'wood',
  'wood',
  'wood',
  'wood',
  'wood',
  'wood',
  'wood',
  'wood',
  'wood',
  'wood',
  'wood',
  'wood',
  'wood',
  'wood',
  'wood',
  'wood',
  'wood',
  'wood',
  'wood',
  'wood',
  'wood',
  'wood',
  'wood',
  'woo

Получается слишком много в пределах одного сектора.

Как это исправить?

**Вопросы:**
    
1. Агент находится внутри прямоугольных границ объекта. Однако это далеко не всегда значит, что агент находится внутри самого объекта. Считать ли в таком случае, что агент видит объект вокруг себя на все 360 градусов? Или считать, что он видит этот объект по максимально возможному углу?

2. На картинке может быть очень много разных объектов. Понятно, что агент не может видеть их все. Значит, их нужно учитывать не все? Тогда какие и как?
    - Стоит ли вводить ограничения по расстоянию?
    - Стоит ли учитывать перекрывающиеся объекты? (Кажется, что когда лес, например, находится за озером его тоже видно)
    - Стоит ли учитывать однотипные объекты в одном и том же направлении?